# Totale

##### setup

In [593]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import altair as alt
import os.path
import getpass
import platform
from pandasql import sqldf
import openpyxl

username = getpass.getuser()

print("Username: ", username)
pathPart1 = "/Users/"
pathPart2 = "/Github/SCGProject/Datasets/"
complePath = pathPart1+username+pathPart2

if platform.system() == "Darwin":
    if(username == "marcovinciguerra"):
        dfCambio = pd.read_csv(
            complePath+"/CorrectedDatasets/dfCambio.csv")

        dfClienti = pd.read_csv(
            complePath+"/CorrectedDatasets/dfClienti.csv")

        dfConsumi = pd.read_csv(
             complePath + "/CorrectedDatasets/dfConsumi.csv")

        dfCostoOrarioConsuntivo = pd.read_csv(
            complePath+"/CorrectedDatasets/dfConsuntivo.csv")

        dfCostoOrarioBudget = pd.read_csv(
            complePath+"CorrectedDatasets/dfCostoOrario.csv")

        dfImpiegoRisorse = pd.read_csv(
         complePath+"/CorrectedDatasets/dfImpiegoRisorse.csv")

        dftuttiClienti = pd.read_csv(
         complePath+"/CorrectedDatasets/dftuttiClienti.csv")

        dfVendite = pd.read_csv(
         complePath+"/CorrectedDatasets/dfVendite.csv")

    elif(username == "davidguzman"):
            pathPart1 = "/Users/"
            pathPart2 = "/documents/Github/SCGProject/Datasets/"
            complePath = pathPart1+username+pathPart2

            dfCambio = pd.read_csv(
                complePath+"/CorrectedDatasets/dfCambio.csv")

            dfClienti = pd.read_csv(
                complePath+"/CorrectedDatasets/dfClienti.csv")

            dfConsumi = pd.read_csv(
                complePath + "/CorrectedDatasets/dfConsumi.csv")

            dfCostoOrarioConsuntivo = pd.read_csv(
                complePath+"/CorrectedDatasets/dfConsuntivo.csv")

            dfCostoOrarioBudget = pd.read_csv(
                complePath+"CorrectedDatasets/dfCostoOrario.csv")

            dfImpiegoRisorse = pd.read_csv(
            complePath+"/CorrectedDatasets/dfImpiegoRisorse.csv")

            dftuttiClienti = pd.read_csv(
            complePath+"/CorrectedDatasets/dftuttiClienti.csv")

            dfVendite = pd.read_csv(
            complePath+"/CorrectedDatasets/dfVendite.csv")

elif platform.system() == "Linux":
    complePath = "/home/alinux/unibg/git_repo/SCGProject"
    pathPart1 = "/Users/"
    pathPart2 = "/documents/Github/SCGProject/Datasets/"
    complePath = pathPart1+username+pathPart2

    dfCambio = pd.read_csv(
        complePath+"/CorrectedDatasets/dfCambio.csv")

    dfClienti = pd.read_csv(
        complePath+"/CorrectedDatasets/dfClienti.csv")

    dfConsumi = pd.read_csv(
        complePath + "/CorrectedDatasets/dfConsumi.csv")

    dfCostoOrarioConsuntivo = pd.read_csv(
        complePath+"/CorrectedDatasets/dfConsuntivo.csv")

    dfCostoOrarioBudget = pd.read_csv(
        complePath+"CorrectedDatasets/dfCostoOrario.csv")

    dfImpiegoRisorse = pd.read_csv(
        complePath+"/CorrectedDatasets/dfImpiegoRisorse.csv")

    dftuttiClienti = pd.read_csv(
        complePath+"/CorrectedDatasets/dftuttiClienti.csv")

    dfVendite = pd.read_csv(
        complePath+"/CorrectedDatasets/dfVendite.csv")



dfCambio["TassoCambioMedio"] = dfCambio["TassoCambioMedio"].str.replace(',','.')


Username:  davidguzman


# Ricavi

In [594]:
dfVendite.rename(columns = {'budget/cons':'budget'}, inplace = True)
venditeConsuntivo = sqldf("SELECT DISTINCT * FROM dfVendite WHERE budget = 'CONSUNTIVO' or budget = 'Consuntivo'")
venditeBudget = sqldf("SELECT DISTINCT * FROM dfVendite WHERE budget = 'BUDGET' or budget = 'Budget'")

In [595]:
print(venditeConsuntivo.shape, venditeBudget.shape)

(1767, 7) (1209, 7)


In [596]:
# join with dfCambio and dfClienti
venditeConsuntivo = sqldf('''SELECT DISTINCT v.NrMovimento, v.budget, v.NrArticolo, v.Quantity, v.ImportoVenditaValutaLocaleTOTALEVENDITA,k.TassoCambioMedio,v.ImportoVenditaValutaLocaleTOTALEVENDITA/k.TassoCambioMedio as TotaleEuro, v.NrOrigine, c.CodCondizioniPagam, c.FattCumulative, c.Valuta
FROM venditeConsuntivo as v join dfClienti as c on v.NrOrigine = c.Nr join dfCambio as k on c.Valuta = k.CodiceValuta
where k.Anno = 'Consuntivo' or k.Anno = 'CONSUNTIVO'
''')

venditeBudget = sqldf('''SELECT DISTINCT v.NrMovimento, v.budget, v.NrArticolo, v.Quantity, v.ImportoVenditaValutaLocaleTOTALEVENDITA,k.TassoCambioMedio,v.ImportoVenditaValutaLocaleTOTALEVENDITA/k.TassoCambioMedio as TotaleEuro, v.NrOrigine, c.CodCondizioniPagam, c.FattCumulative, c.Valuta
FROM venditeBudget as v join dfClienti as c on v.NrOrigine = c.Nr join dfCambio as k on c.Valuta = k.CodiceValuta
where k.Anno = 'Budget' or k.Anno = 'BUDGET'
''')

In [597]:
print(venditeConsuntivo.shape, venditeBudget.shape)

(1767, 11) (1209, 11)


In [598]:
sumVenditeConsuntivo = sqldf('select sum(TotaleEuro) as sumVenditaTotale from venditeConsuntivo')
sumVenditeBudget = sqldf('select sum(TotaleEuro) as sumVenditaTotale from venditeBudget')
print(sumVenditeConsuntivo)
print(sumVenditeBudget)
print(sumVenditeConsuntivo-sumVenditeBudget) #questo è lo scostamento totale dei ricavi

   sumVenditaTotale
0     507842.851413
   sumVenditaTotale
0     395335.892443
   sumVenditaTotale
0      112506.95897


# Costi

Materie prime

In [599]:
dfConsumi.rename(columns = {'Budget/cons':'budget'}, inplace = True)
consumiConsuntivo = sqldf("SELECT DISTINCT * FROM dfConsumi WHERE budget = 'CONSUNTIVO'")
consumiBudget = sqldf("SELECT DISTINCT * FROM dfConsumi WHERE budget = 'BUDGET'")
sumConsumiConsuntivo = sqldf('select sum(ImportoCostoTOTALE) from consumiConsuntivo')
sumConsumiBudget = sqldf('select sum(ImportoCostoTOTALE) from consumiBudget')
print(sumConsumiConsuntivo)
print(sumConsumiBudget)
print(sumConsumiConsuntivo-sumConsumiBudget) #questo è lo scostamento totale della materia prima

   sum(ImportoCostoTOTALE)
0                 289639.5
   sum(ImportoCostoTOTALE)
0                192906.47
   sum(ImportoCostoTOTALE)
0                 96733.03


Lavorazioni interne

In [600]:
impiegoConsuntivo = sqldf("SELECT DISTINCT * FROM dfImpiegoRisorse WHERE budgetConsuntivo = 'CONSUNTIVO' or budgetConsuntivo = 'Consuntivo'")
print(impiegoConsuntivo.shape)
#join con i costi orari
impiegoConsuntivo  = sqldf('''SELECT DISTINCT i.NrArticolo,i.budgetConsuntivo,i.NrOrdineProduzione,i.Descrizione,i.NrAreaProduzione,i.Risorsa,i.QuantitydiOutput,i.TempoRisorsa,c.CostoOrario
FROM impiegoConsuntivo as i join dfCostoOrarioConsuntivo as c 
on c.AreaProduzione = i.NrAreaProduzione and c.Risorsa = i.Risorsa''')
print(impiegoConsuntivo.shape)
# ci sono alcune quantità di output nulle, per gestire le sommiamo alle tuple con tutte le altre cose uguali
impiegoConsuntivo = sqldf('''select NrArticolo,budgetConsuntivo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,sum(TempoRisorsa) as TempoRisorsa,CostoOrario, sum(QuantitydiOutput) as QuantitydiOutput
from impiegoConsuntivo
group by NrArticolo,budgetConsuntivo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,CostoOrario
order by NrArticolo
''')
impiegoConsuntivo

(3791, 9)
(3723, 9)


,NrArticolo,budgetConsuntivo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,TempoRisorsa,CostoOrario,QuantitydiOutput
0,ART0000018,CONSUNTIVO,ODP19-05998,Controllo qualità,CQ,RIS16,0.0000,15,2.0
1,ART0000018,CONSUNTIVO,ODP19-05998,Fresatura,A30,RIS5,1.0000,50,2.0
2,ART0000018,CONSUNTIVO,ODP19-05998,Tornitura,A20,RIS2,1.0000,80,2.0
3,ART0000041,CONSUNTIVO,ODP19-05998,Controllo qualità,CQ,RIS16,0.0000,15,4.0
4,ART0000041,CONSUNTIVO,ODP19-05998,Fresatura,A30,RIS1,0.0001,80,4.0
...,...,...,...,...,...,...,...,...,...
3012,ART0005136,CONSUNTIVO,ODP19-06532,Fresatura,A30,RIS14,1.2500,50,3.0
3013,ART0005136,CONSUNTIVO,ODP19-06533,Controllo qualità,CQ,RIS16,0.0000,15,4.0
3014,ART0005136,CONSUNTIVO,ODP19-06533,Tornitura,A20,RIS13,0.5000,80,4.0
3015,ART0005136,CONSUNTIVO,ODP19-06688,Controllo qualità,CQ,RIS16,0.0000,15,9.0


In [601]:
impiegoBudget = sqldf("SELECT DISTINCT * FROM dfImpiegoRisorse WHERE budgetConsuntivo = 'BUDGET' or budgetConsuntivo = 'Budget'")
#join con i costi orari
print(impiegoBudget.shape)
impiegoBudget  = sqldf('''SELECT DISTINCT i.NrArticolo,i.budgetConsuntivo,i.NrOrdineProduzione,i.Descrizione,i.NrAreaProduzione,i.Risorsa,i.QuantitydiOutput,i.TempoRisorsa,c.CostoOrario
FROM impiegoBudget as i join dfCostoOrarioBudget as c 
on c.AreaProduzione = i.NrAreaProduzione and c.Risorsa = i.Risorsa''')
print(impiegoBudget.shape)
# righe a consuntivo sono di più che a budget e la somma dei 2 non corrisponde al numero di righe della tabella impiego risorse

# ci sono alcune quantità di output nulle, per gestire le sommiamo alle tuple con tutte le altre cose uguali
impiegoBudget = sqldf('''select NrArticolo,budgetConsuntivo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,sum(TempoRisorsa) as TempoRisorsa,CostoOrario, sum(QuantitydiOutput) as QuantitydiOutput
from impiegoBudget
group by NrArticolo,budgetConsuntivo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,CostoOrario
order by NrArticolo
''')
impiegoBudget


(2473, 9)
(2450, 9)


,NrArticolo,budgetConsuntivo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,TempoRisorsa,CostoOrario,QuantitydiOutput
0,ART0000018,BUDGET,ODP17-05342,Controllo qualità,CQ,RIS16,0.00,12.75,2.0
1,ART0000018,BUDGET,ODP17-05342,Fresatura,A30,RIS7,0.75,51.00,2.0
2,ART0000018,BUDGET,ODP17-05342,Tornitura,A20,RIS2,1.00,68.00,2.0
3,ART0000041,BUDGET,ODP17-05342,Controllo qualità,CQ,RIS16,0.00,12.75,4.0
4,ART0000041,BUDGET,ODP17-05342,Fresatura,A30,RIS7,1.25,51.00,4.0
...,...,...,...,...,...,...,...,...,...
2006,ART0005136,BUDGET,ODP17-15077,Controllo qualità,CQ,RIS16,0.00,12.75,10.0
2007,ART0005136,BUDGET,ODP17-15077,Tornitura,A20,RIS14,1.25,51.00,10.0
2008,ART0005136,BUDGET,ODP17-16230,Brocciatura / stozzatrice,A33,RIS13,0.25,42.50,1.0
2009,ART0005136,BUDGET,ODP17-16230,Controllo qualità,CQ,RIS16,0.00,12.75,1.0


In [602]:
impiegoBudget

sqldf('''select NrArticolo, NrOrdineProduzione, Descrizione, NrAreaProduzione, Risorsa, count(*)
from impiegoBudget 
group by NrArticolo, NrOrdineProduzione, Descrizione, NrAreaProduzione, Risorsa
order by count(*) desc''')

sqldf('''select NrArticolo, Descrizione, NrAreaProduzione, Risorsa, count(*)
from impiegoBudget 
group by NrArticolo, Descrizione, NrAreaProduzione, Risorsa
order by count(*) desc''')

,NrArticolo,Descrizione,NrAreaProduzione,Risorsa,count(*)
0,ART0005136,Controllo qualità,CQ,RIS16,19
1,ART0005136,Tornitura,A20,RIS2,13
2,ART0000795,Controllo qualità,CQ,RIS16,11
3,ART0000762,Controllo qualità,CQ,RIS16,9
4,ART0000782,Controllo qualità,CQ,RIS16,8
...,...,...,...,...,...
1395,ART0005136,Brocciatura / stozzatrice,A33,RIS2,1
1396,ART0005136,Foratura/Maschiatura,A32,RIS9,1
1397,ART0005136,Fresatura,A30,RIS1,1
1398,ART0005136,Fresatura,A30,RIS13,1


# Scostamento totale

In [603]:
sumImpiegoConsuntivo = sqldf(
    'SELECT sum(TempoRisorsa*CostoOrario) FROM impiegoConsuntivo WHERE TempoRisorsa>=0')
sumImpiegoBudget = sqldf(
    'SELECT sum(TempoRisorsa*CostoOrario) FROM impiegoBudget WHERE TempoRisorsa>=0')
print(sumImpiegoConsuntivo)
print(sumImpiegoBudget)
print(sumImpiegoConsuntivo-sumImpiegoBudget)

   sum(TempoRisorsa*CostoOrario)
0                    359238.2962
   sum(TempoRisorsa*CostoOrario)
0                   194521.50346
   sum(TempoRisorsa*CostoOrario)
0                   164716.79274


In [604]:
print(sumVenditeConsuntivo.values[0][0])
print(sumImpiegoConsuntivo.values[0][0]+sumConsumiConsuntivo.values[0][0])
print(sumVenditeConsuntivo.values[0][0]-sumImpiegoConsuntivo.values[0][0]-sumConsumiConsuntivo.values[0][0])
print(sumVenditeBudget.values[0][0])
print(sumImpiegoBudget.values[0][0]+sumConsumiBudget.values[0][0])
print(sumVenditeBudget.values[0][0]-sumImpiegoBudget.values[0][0]-sumConsumiBudget.values[0][0])

507842.8514127266
648877.7962
-141034.94478727342
395335.8924431994
387427.97346000024
7907.918983199197


Senza vincoli su tempi negativi

**non cambia perché per ogni tempo negativo esiste un tempo positivo uguale in modulo che, al sommare i tempi delle stesse attività, si annullano a vicenda**

In [605]:
sumImpiegoConsuntivo = sqldf(
    'SELECT sum(TempoRisorsa*CostoOrario) FROM impiegoConsuntivo')
sumImpiegoBudget = sqldf(
    'SELECT sum(TempoRisorsa*CostoOrario) FROM impiegoBudget')
print(sumImpiegoConsuntivo)
print(sumImpiegoBudget)
print(sumImpiegoConsuntivo-sumImpiegoBudget)

   sum(TempoRisorsa*CostoOrario)
0                    359215.7962
   sum(TempoRisorsa*CostoOrario)
0                   194521.50346
   sum(TempoRisorsa*CostoOrario)
0                   164694.29274


In [606]:
print(sumVenditeConsuntivo.values[0][0])
print(sumImpiegoConsuntivo.values[0][0]+sumConsumiConsuntivo.values[0][0])
print(sumVenditeConsuntivo.values[0][0]-sumImpiegoConsuntivo.values[0][0]-sumConsumiConsuntivo.values[0][0])
print(sumVenditeBudget.values[0][0])
print(sumImpiegoBudget.values[0][0]+sumConsumiBudget.values[0][0])
print(sumVenditeBudget.values[0][0]-sumImpiegoBudget.values[0][0]-sumConsumiBudget.values[0][0])
print(-141012.44478727342 - 7907.918983199197)


507842.8514127266
648855.2962
-141012.44478727342
395335.8924431994
387427.97346000024
7907.918983199197
-148920.3637704726


# Socostamento di volume, impiego e prezzo per le Lavorazioni

In [607]:
# Creazione colonna ImpiegoUnitario
impiegoConsuntivo = sqldf(''' select NrArticolo, budgetConsuntivo, NrOrdineProduzione, Descrizione, NrAreaProduzione, Risorsa, TempoRisorsa, TempoRisorsa/QuantitydiOutput as ImpiegoUnitario, CostoOrario ,QuantitydiOutput
from impiegoConsuntivo''')
impiegoConsuntivo.drop(['NrOrdineProduzione'], axis=1, inplace=True) #Non cambia la quantità di righe!
impiegoConsuntivo

,NrArticolo,budgetConsuntivo,Descrizione,NrAreaProduzione,Risorsa,TempoRisorsa,ImpiegoUnitario,CostoOrario,QuantitydiOutput
0,ART0000018,CONSUNTIVO,Controllo qualità,CQ,RIS16,0.0000,0.000000,15,2.0
1,ART0000018,CONSUNTIVO,Fresatura,A30,RIS5,1.0000,0.500000,50,2.0
2,ART0000018,CONSUNTIVO,Tornitura,A20,RIS2,1.0000,0.500000,80,2.0
3,ART0000041,CONSUNTIVO,Controllo qualità,CQ,RIS16,0.0000,0.000000,15,4.0
4,ART0000041,CONSUNTIVO,Fresatura,A30,RIS1,0.0001,0.000025,80,4.0
...,...,...,...,...,...,...,...,...,...
3012,ART0005136,CONSUNTIVO,Fresatura,A30,RIS14,1.2500,0.416667,50,3.0
3013,ART0005136,CONSUNTIVO,Controllo qualità,CQ,RIS16,0.0000,0.000000,15,4.0
3014,ART0005136,CONSUNTIVO,Tornitura,A20,RIS13,0.5000,0.125000,80,4.0
3015,ART0005136,CONSUNTIVO,Controllo qualità,CQ,RIS16,0.0000,0.000000,15,9.0


In [608]:
sqldf('select sum(ImpiegoUnitario*CostoOrario*QuantitydiOutput) from impiegoConsuntivo') 

,sum(ImpiegoUnitario*CostoOrario*QuantitydiOutput)
0,329701.5902


In [609]:
sqldf('select sum(TempoRisorsa*CostoOrario) from impiegoConsuntivo') 

,sum(TempoRisorsa*CostoOrario)
0,359215.7962


Notiamo che sono diversi, e diminiuisce se utilizziamo 'ImpiegoUnitario' questo è dovuto a:

In [610]:
x = sqldf('''select NrArticolo,budgetConsuntivo,Descrizione,NrAreaProduzione,Risorsa,TempoRisorsa,ImpiegoUnitario,CostoOrario,QuantitydiOutput, TempoRisorsa*CostoOrario, ImpiegoUnitario*CostoOrario*QuantitydiOutput
from impiegoConsuntivo''')
#in conclusione, i totali cambiano perché ci sono quantità di output nulle! quindi alcuni impieghi diventano nan
# TODO IDEA PER RISOLVERE: fare avg di CostoOrario e sum TempoRisorsa, QuantitydiOutput in quei casi in cui la descrizione è uguale ma la risorsa è diversa, il totale dovrebbe essere uguale

Avg out the null values

In [611]:
impiegoConsuntivo = sqldf(''' select NrArticolo, budgetConsuntivo, Descrizione, NrAreaProduzione, sum(TempoRisorsa) as TempoRisorsa, avg(CostoOrario) as CostoOrario, sum(QuantitydiOutput) as QuantitydiOutput
from impiegoConsuntivo
group by NrArticolo, budgetConsuntivo, Descrizione, NrAreaProduzione
''')

vol = sqldf('select sum(QuantitydiOutput) as Volume from impiegoConsuntivo')

impiegoConsuntivo = sqldf(''' select NrArticolo, budgetConsuntivo, Descrizione, NrAreaProduzione, TempoRisorsa, (TempoRisorsa/Volume) as ImpiegoUnitario, CostoOrario, Volume ,QuantitydiOutput
from impiegoConsuntivo, vol
''')
impiegoConsuntivo


,NrArticolo,budgetConsuntivo,Descrizione,NrAreaProduzione,TempoRisorsa,ImpiegoUnitario,CostoOrario,Volume,QuantitydiOutput
0,ART0000018,CONSUNTIVO,Controllo qualità,CQ,0.0000,0.000000e+00,15.000000,25950.0,2.0
1,ART0000018,CONSUNTIVO,Fresatura,A30,1.0000,3.853565e-05,50.000000,25950.0,2.0
2,ART0000018,CONSUNTIVO,Tornitura,A20,1.0000,3.853565e-05,80.000000,25950.0,2.0
3,ART0000041,CONSUNTIVO,Controllo qualità,CQ,0.0000,0.000000e+00,15.000000,25950.0,4.0
4,ART0000041,CONSUNTIVO,Fresatura,A30,0.0001,3.853565e-09,80.000000,25950.0,4.0
...,...,...,...,...,...,...,...,...,...
1281,ART0005136,CONSUNTIVO,Saldatura,A11,21.8550,8.421965e-04,50.000000,25950.0,44.0
1282,ART0005136,CONSUNTIVO,Taglio,A10,12.0000,4.624277e-04,60.000000,25950.0,1.0
1283,ART0005136,CONSUNTIVO,Tornitura,A20,143.9091,5.545630e-03,75.903614,25950.0,539.0
1284,ART0005136,CONSUNTIVO,Tornitura (chiedere a Massimo),A20,3.7500,1.445087e-04,80.000000,25950.0,1.0


In [612]:
sqldf('select sum(TempoRisorsa*CostoOrario) from impiegoConsuntivo') # si mantiene quasi uguale!!

,sum(TempoRisorsa*CostoOrario)
0,359425.545873


In [613]:
sqldf('select sum(ImpiegoUnitario*CostoOrario*Volume) from impiegoConsuntivo') 

,sum(ImpiegoUnitario*CostoOrario*Volume)
0,359425.545873


la piccola differenza è dovuta alla restante parte di righe con output nulle e senza altre righe con output non nullo da sommare

In [614]:
sqldf('select * from impiegoConsuntivo where ImpiegoUnitario is null') 

,NrArticolo,budgetConsuntivo,Descrizione,NrAreaProduzione,TempoRisorsa,ImpiegoUnitario,CostoOrario,Volume,QuantitydiOutput


#### ripetiamo per budget

In [615]:
impiegoBudget = sqldf(''' select NrArticolo, budgetConsuntivo, Descrizione, NrAreaProduzione, sum(TempoRisorsa) as TempoRisorsa, avg(CostoOrario) as CostoOrario, sum(QuantitydiOutput) as QuantitydiOutput
from impiegoBudget
group by NrArticolo, budgetConsuntivo, Descrizione, NrAreaProduzione
''')
vol = sqldf('select sum(QuantitydiOutput) as Volume from impiegoBudget')

impiegoBudget = sqldf(''' select NrArticolo, budgetConsuntivo, Descrizione, NrAreaProduzione, TempoRisorsa, (TempoRisorsa/Volume) as ImpiegoUnitario, CostoOrario, Volume ,QuantitydiOutput
from impiegoBudget, vol
''')
impiegoBudget

,NrArticolo,budgetConsuntivo,Descrizione,NrAreaProduzione,TempoRisorsa,ImpiegoUnitario,CostoOrario,Volume,QuantitydiOutput
0,ART0000018,BUDGET,Controllo qualità,CQ,0.00,0.000000,12.750000,15768.5,2.0
1,ART0000018,BUDGET,Fresatura,A30,0.75,0.000048,51.000000,15768.5,2.0
2,ART0000018,BUDGET,Tornitura,A20,1.00,0.000063,68.000000,15768.5,2.0
3,ART0000041,BUDGET,Controllo qualità,CQ,0.00,0.000000,12.750000,15768.5,4.0
4,ART0000041,BUDGET,Fresatura,A30,1.25,0.000079,51.000000,15768.5,4.0
...,...,...,...,...,...,...,...,...,...
1222,ART0005136,BUDGET,Foratura/Maschiatura,A32,3.00,0.000190,42.500000,15768.5,0.0
1223,ART0005136,BUDGET,Fresatura,A30,21.33,0.001353,47.600000,15768.5,24.0
1224,ART0005136,BUDGET,Montaggio,A40,8.25,0.000523,42.500000,15768.5,0.0
1225,ART0005136,BUDGET,Taglio spezzoni,A20,4.50,0.000285,68.000000,15768.5,29.0


In [616]:
sqldf('select sum(TempoRisorsa*CostoOrario) from impiegoBudget') # si mantiene quasi uguale!!

,sum(TempoRisorsa*CostoOrario)
0,194739.149842


In [617]:
sqldf('select sum(ImpiegoUnitario*CostoOrario*Volume) from impiegoBudget') 

,sum(ImpiegoUnitario*CostoOrario*Volume)
0,194739.149842


In [618]:
sqldf('select * from impiegoBudget where ImpiegoUnitario is null') 

,NrArticolo,budgetConsuntivo,Descrizione,NrAreaProduzione,TempoRisorsa,ImpiegoUnitario,CostoOrario,Volume,QuantitydiOutput


#### Merge/Join

In [619]:
impiegoConsuntivo.drop(['budgetConsuntivo'], axis=1, inplace=True)
impiegoBudget.drop(['budgetConsuntivo'], axis=1, inplace=True)
x = pd.merge(impiegoConsuntivo, impiegoBudget, on=['NrArticolo', 'Descrizione', 'NrAreaProduzione'], how='outer')
x

,NrArticolo,Descrizione,NrAreaProduzione,TempoRisorsa_x,ImpiegoUnitario_x,CostoOrario_x,Volume_x,QuantitydiOutput_x,TempoRisorsa_y,ImpiegoUnitario_y,CostoOrario_y,Volume_y,QuantitydiOutput_y
0,ART0000018,Controllo qualità,CQ,0.0000,0.000000e+00,15.0,25950.0,2.0,0.00,0.000000,12.75,15768.5,2.0
1,ART0000018,Fresatura,A30,1.0000,3.853565e-05,50.0,25950.0,2.0,0.75,0.000048,51.00,15768.5,2.0
2,ART0000018,Tornitura,A20,1.0000,3.853565e-05,80.0,25950.0,2.0,1.00,0.000063,68.00,15768.5,2.0
3,ART0000041,Controllo qualità,CQ,0.0000,0.000000e+00,15.0,25950.0,4.0,0.00,0.000000,12.75,15768.5,4.0
4,ART0000041,Fresatura,A30,0.0001,3.853565e-09,80.0,25950.0,4.0,1.25,0.000079,51.00,15768.5,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1345,ART0004904,Tornitura,A20,NaN,NaN,NaN,NaN,NaN,1.75,0.000111,68.00,15768.5,4.0
1346,ART0004944,Tornitura,A20,NaN,NaN,NaN,NaN,NaN,1.75,0.000111,68.00,15768.5,8.0
1347,ART0005091,Fresatura,A30,NaN,NaN,NaN,NaN,NaN,1.00,0.000063,42.50,15768.5,4.0
1348,ART0005136,Foratura/Maschiatura,A32,NaN,NaN,NaN,NaN,NaN,3.00,0.000190,42.50,15768.5,0.0


In [620]:
c = sqldf('select sum(ImpiegoUnitario_x*CostoOrario_x*Volume_x) from x').iloc[0][0]
b = sqldf('select sum(ImpiegoUnitario_y*CostoOrario_y*Volume_y) from x').iloc[0][0]
print(c)
print(b)
print(c-b)
# non altera i totali!!

359425.54587289935
194739.14984166672
164686.39603123264


In [621]:
#Budget Standard
BudgetStandardLav = sqldf('select sum(ImpiegoUnitario_y*CostoOrario_y*Volume_y) from x').iloc[0][0]
# Impiego Standard (volume a consuntivo)
ImpiegoStandardLav = sqldf('select sum(ImpiegoUnitario_y*CostoOrario_y*Volume_x) from x').iloc[0][0]
# Impiego Effettivo (volume e impiego a consuntivo)
ImpiegoEffettivoLav=sqldf('select sum(ImpiegoUnitario_x*CostoOrario_y*Volume_x) from x').iloc[0][0]
# Budget Effettivo (tutto a consuntivo)
BudgetEffettivoLav=sqldf('select sum(ImpiegoUnitario_x*CostoOrario_x*Volume_x) from x').iloc[0][0]
print(BudgetStandardLav)
print(ImpiegoStandardLav)
print(ImpiegoEffettivoLav)
print(BudgetEffettivoLav)

194739.14984166672
305606.8516222373
281906.66007
359425.54587289935


In [622]:
#Scostamenti
ScostamentoVolumeLav = ImpiegoStandardLav-b
ScostamentoImpiegoLav = ImpiegoEffettivoLav-ImpiegoStandardLav
ScostamentoPrezzoLav = BudgetEffettivoLav-ImpiegoEffettivoLav
print(ScostamentoVolumeLav)
print(ScostamentoImpiegoLav)
print(ScostamentoPrezzoLav)
print(ScostamentoVolumeLav+ScostamentoImpiegoLav+ScostamentoPrezzoLav) # uguale allo scostamento totale per le lavorazioni!!!

110867.70178057058
-23700.191552237317
77518.88580289937
164686.39603123264


# Socostamento di volume, impiego e prezzo per le materie prime

#### without group by

In [623]:
sqldf('select CodiceMP, count(*) as nmp from consumiConsuntivo group by CodiceMP order by nmp desc')

,CodiceMP,nmp
0,ART0005176,116
1,ART0005216,64
2,ART0003831,64
3,ART0005139,58
4,ART0001387,49
...,...,...
625,ART0001075,1
626,ART0001068,1
627,ART0001066,1
628,ART0000809,1


In [624]:
consumiConsuntivoUnit = sqldf('''select distinct NrMovimento, CodiceMP, NrArticolo, QuantityMPImpiegata, ImportoCostoTOTALE/QuantityMPImpiegata as CostoUnitario, ImportoCostoTOTALE 
from consumiConsuntivo
order by CodiceMP''')
consumiBudgetUnit = sqldf('''select distinct NrMovimento, CodiceMP, NrArticolo, QuantityMPImpiegata, ImportoCostoTOTALE/QuantityMPImpiegata as CostoUnitario, ImportoCostoTOTALE 
from consumiBudget
order by CodiceMP''')

quick check, numbers remain the same!

In [625]:
print(sqldf('select sum(ImportoCostoTOTALE) from consumiConsuntivoUnit'))
print(sqldf('select sum(ImportoCostoTOTALE) from consumiBudgetUnit'))

   sum(ImportoCostoTOTALE)
0                 289639.5
   sum(ImportoCostoTOTALE)
0                192906.47


In [626]:
print(sqldf('select sum(CostoUnitario*QuantityMPImpiegata) from consumiConsuntivoUnit'))
print(sqldf('select sum(CostoUnitario*QuantityMPImpiegata) from consumiBudgetUnit'))

   sum(CostoUnitario*QuantityMPImpiegata)
0                                289639.5
   sum(CostoUnitario*QuantityMPImpiegata)
0                               192906.47


In [627]:
#non uso NrMovimento come chiave perchè è unico per ogni riga e cambia da budget a consuntivo
ConsumiUnit = pd.merge(consumiConsuntivoUnit,consumiBudgetUnit, on = ['CodiceMP','NrArticolo'], how = 'outer')
ConsumiUnit
# se non facciamo qualche group by a priori si sdoppiano le righe

,NrMovimento_x,CodiceMP,NrArticolo,QuantityMPImpiegata_x,CostoUnitario_x,ImportoCostoTOTALE_x,NrMovimento_y,QuantityMPImpiegata_y,CostoUnitario_y,ImportoCostoTOTALE_y
0,53273.0,ART0000771,ART0005136,1.0,230.950000,230.95,NaN,NaN,NaN,NaN
1,53274.0,ART0000809,ART0005136,2.0,561.350000,1122.70,NaN,NaN,NaN,NaN
2,52602.0,ART0001066,ART0003099,1064.0,0.001870,1.99,NaN,NaN,NaN,NaN
3,62498.0,ART0001068,ART0004671,190.0,0.036000,6.84,NaN,NaN,NaN,NaN
4,47038.0,ART0001073,ART0002240,184.0,0.086685,15.95,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
10466,NaN,ART0005913,ART0002191,NaN,NaN,NaN,34425.0,40.0,0.339250,13.57
10467,NaN,ART0005917,ART0001610,NaN,NaN,NaN,34469.0,972.0,0.000597,0.58
10468,NaN,ART0005930,ART0000229,NaN,NaN,NaN,34993.0,1560.0,0.005064,7.90
10469,NaN,ART0005935,ART0004675,NaN,NaN,NaN,35033.0,225.0,0.014311,3.22


In [628]:
sqldf('select sum(ImportoCostoTOTALE_x) from ConsumiUnit')

,sum(ImportoCostoTOTALE_x)
0,1067468.83


#### Group by CodiceMP

se non facciamo qualche group by a priori si sdoppiano le righe, proviamo prima con CodiceMP

In [629]:
print(sqldf('select sum(QuantityMPImpiegata) as Volume from consumiConsuntivoUnit'))
print(sqldf('select sum(QuantityMPImpiegata) as Volume from consumiBudgetUnit'))

      Volume
0  938999.16
     Volume
0  550834.0


In [630]:
consumiConsuntivoUnitByMP = sqldf('''select CodiceMP, NrArticolo,938999.16 as Volume, sum(QuantityMPImpiegata) as QuantityMPImpiegata,sum(ImportoCostoTOTALE)/sum(QuantityMPImpiegata) as CostoUnitario , sum(ImportoCostoTOTALE) as ImportoCostoTOTALE
from consumiConsuntivoUnit      
group by CodiceMP, NrArticolo''')
ConsumiBudgetUnitByMP = sqldf('''select CodiceMP, NrArticolo,550834.0 as Volume, sum(QuantityMPImpiegata) as QuantityMPImpiegata,sum(ImportoCostoTOTALE)/sum(QuantityMPImpiegata) as CostoUnitario , sum(ImportoCostoTOTALE) as ImportoCostoTOTALE
from consumiBudgetUnit      
group by CodiceMP, NrArticolo''')
consumiConsuntivoUnitByMP

,CodiceMP,NrArticolo,Volume,QuantityMPImpiegata,CostoUnitario,ImportoCostoTOTALE
0,ART0000771,ART0005136,938999.16,1.0,230.950000,230.95
1,ART0000809,ART0005136,938999.16,2.0,561.350000,1122.70
2,ART0001066,ART0003099,938999.16,1064.0,0.001870,1.99
3,ART0001068,ART0004671,938999.16,190.0,0.036000,6.84
4,ART0001073,ART0002240,938999.16,184.0,0.086685,15.95
...,...,...,...,...,...,...
1074,ART0007130,ART0001228,938999.16,15.0,1.500000,22.50
1075,ART0007133,ART0005136,938999.16,3.0,0.000000,0.00
1076,ART0007141,ART0002191,938999.16,2.0,6.500000,13.00
1077,ART0007148,ART0002619,938999.16,2.0,30.000000,60.00


In [631]:
consumiConsuntivoUnitByMP = sqldf('''select CodiceMP, NrArticolo, Volume, (QuantityMPImpiegata/Volume) as Impiego, CostoUnitario, ImportoCostoTOTALE
from consumiConsuntivoUnitByMP''')
ConsumiBudgetUnitByMP = sqldf('''select CodiceMP, NrArticolo, Volume, (QuantityMPImpiegata/Volume) as Impiego, CostoUnitario, ImportoCostoTOTALE
from ConsumiBudgetUnitByMP''')
consumiConsuntivoUnitByMP

,CodiceMP,NrArticolo,Volume,Impiego,CostoUnitario,ImportoCostoTOTALE
0,ART0000771,ART0005136,938999.16,0.000001,230.950000,230.95
1,ART0000809,ART0005136,938999.16,0.000002,561.350000,1122.70
2,ART0001066,ART0003099,938999.16,0.001133,0.001870,1.99
3,ART0001068,ART0004671,938999.16,0.000202,0.036000,6.84
4,ART0001073,ART0002240,938999.16,0.000196,0.086685,15.95
...,...,...,...,...,...,...
1074,ART0007130,ART0001228,938999.16,0.000016,1.500000,22.50
1075,ART0007133,ART0005136,938999.16,0.000003,0.000000,0.00
1076,ART0007141,ART0002191,938999.16,0.000002,6.500000,13.00
1077,ART0007148,ART0002619,938999.16,0.000002,30.000000,60.00


In [632]:
print(sqldf('select sum(ImportoCostoTOTALE) from consumiConsuntivoUnitByMP'))
print(sqldf('select sum(Volume*Impiego*CostoUnitario) from consumiConsuntivoUnitByMP'))
print(sqldf('select sum(ImportoCostoTOTALE) from ConsumiBudgetUnitByMP'))
print(sqldf('select sum(Volume*Impiego*CostoUnitario) from ConsumiBudgetUnitByMP'))
# non si perdono dei decimali 

   sum(ImportoCostoTOTALE)
0                 289639.5
   sum(Volume*Impiego*CostoUnitario)
0                           289639.5
   sum(ImportoCostoTOTALE)
0                192906.47
   sum(Volume*Impiego*CostoUnitario)
0                          192906.47


##### Scostamenti

In [633]:
print(consumiConsuntivoUnitByMP.shape, ConsumiBudgetUnitByMP.shape)

(1079, 6) (937, 6)


In [634]:
ConsumiUnit = pd.merge(consumiConsuntivoUnitByMP,ConsumiBudgetUnitByMP, on = ['CodiceMP','NrArticolo'], how = 'outer')
ConsumiUnit

,CodiceMP,NrArticolo,Volume_x,Impiego_x,CostoUnitario_x,ImportoCostoTOTALE_x,Volume_y,Impiego_y,CostoUnitario_y,ImportoCostoTOTALE_y
0,ART0000771,ART0005136,938999.16,0.000001,230.950000,230.95,NaN,NaN,NaN,NaN
1,ART0000809,ART0005136,938999.16,0.000002,561.350000,1122.70,NaN,NaN,NaN,NaN
2,ART0001066,ART0003099,938999.16,0.001133,0.001870,1.99,NaN,NaN,NaN,NaN
3,ART0001068,ART0004671,938999.16,0.000202,0.036000,6.84,NaN,NaN,NaN,NaN
4,ART0001073,ART0002240,938999.16,0.000196,0.086685,15.95,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1310,ART0005913,ART0002191,NaN,NaN,NaN,NaN,550834.0,0.000073,0.339250,13.57
1311,ART0005917,ART0001610,NaN,NaN,NaN,NaN,550834.0,0.001765,0.000597,0.58
1312,ART0005930,ART0000229,NaN,NaN,NaN,NaN,550834.0,0.002832,0.005064,7.90
1313,ART0005935,ART0004675,NaN,NaN,NaN,NaN,550834.0,0.000408,0.014311,3.22


In [635]:
BudgetEffettivoMP = sqldf('select sum(Volume_x*Impiego_x*CostoUnitario_x) from ConsumiUnit').iloc[0][0]
ImpiegoEffettivoMP = sqldf('select sum(Volume_x*Impiego_x*CostoUnitario_y) from ConsumiUnit').iloc[0][0]
ImpiegoStandardMP = sqldf('select sum(Volume_x*Impiego_y*CostoUnitario_y) from ConsumiUnit').iloc[0][0]
BudgetStandardMP = sqldf('select sum(Volume_y*Impiego_y*CostoUnitario_y) from ConsumiUnit').iloc[0][0]
ScostamentoVolumeMP =  ImpiegoStandardMP - BudgetStandardMP
ScostamentoImpiegoMP = ImpiegoEffettivoMP-ImpiegoStandardMP
ScostamentoPrezzoMP = BudgetEffettivoMP-ImpiegoEffettivoMP 

In [636]:
print(BudgetEffettivoMP)
print(ImpiegoEffettivoMP)
print(ImpiegoStandardMP)
print(BudgetStandardMP)
print(ScostamentoVolumeMP)
print(ScostamentoImpiegoMP)
print(ScostamentoPrezzoMP)
print(ScostamentoVolumeMP+ScostamentoImpiegoMP+ScostamentoPrezzoMP) 

289639.5000000002
273100.71825651696
313091.95489209157
192906.47
120185.48489209157
-39991.236635574605
16538.78174348321
96733.03000000017


# Socostamento di volume, mix e prezzo per i ricavi

 group by NrArticolo dei ricavi

In [637]:
venditeConsuntivoArt = sqldf('''select NrArticolo, sum(Quantity) as Quantity, sum(TotaleEuro) as TotaleEuro
        from venditeConsuntivo
        group by NrArticolo''')
venditeConsuntivoArt

,NrArticolo,Quantity,TotaleEuro
0,ART0000018,2,91.720170
1,ART0000041,4,400.233470
2,ART0000042,4,600.350204
3,ART0000043,2,250.145918
4,ART0000115,3,1095.000000
...,...,...,...
408,ART0005109,9,210.122572
409,ART0005110,9,435.253898
410,ART0005111,7,204.285833
411,ART0005112,7,262.653214


In [638]:
venditeBudgetArt = sqldf('''select NrArticolo, sum(Quantity) as Quantity, sum(TotaleEuro) as TotaleEuro
        from venditeBudget
        group by NrArticolo''')
venditeBudgetArt

,NrArticolo,Quantity,TotaleEuro
0,ART0000018,2,85.380894
1,ART0000041,4,360.497107
2,ART0000042,4,569.205958
3,ART0000043,2,218.195617
4,ART0000115,12,1965.000000
...,...,...,...
408,ART0005109,4,106.251779
409,ART0005110,2,94.867660
410,ART0005111,2,66.407362
411,ART0005112,2,85.380894


##### calcolo del delta dei volumi 

In [639]:

vendite = pd.merge(venditeConsuntivoArt,venditeBudgetArt, on = ['NrArticolo'], how = 'outer')
vendite

,NrArticolo,Quantity_x,TotaleEuro_x,Quantity_y,TotaleEuro_y
0,ART0000018,2,91.720170,2,85.380894
1,ART0000041,4,400.233470,4,360.497107
2,ART0000042,4,600.350204,4,569.205958
3,ART0000043,2,250.145918,2,218.195617
4,ART0000115,3,1095.000000,12,1965.000000
...,...,...,...,...,...
408,ART0005109,9,210.122572,4,106.251779
409,ART0005110,9,435.253898,2,94.867660
410,ART0005111,7,204.285833,2,66.407362
411,ART0005112,7,262.653214,2,85.380894


In [640]:
sqldf('''select sum(TotaleEuro_x) from vendite''')

,sum(TotaleEuro_x)
0,507842.851413


In [641]:
sqldf('''select sum(TotaleEuro_y) from vendite''')

,sum(TotaleEuro_y)
0,395335.892443


In [642]:
sqldf('''select NrArticolo, Quantity_x-Quantity_y as DiffQuantity
        from vendite
        order by DiffQuantity desc
        ''')

,NrArticolo,DiffQuantity
0,ART0005136,1046
1,ART0000748,291
2,ART0001575,200
3,ART0004397,114
4,ART0000229,113
...,...,...
408,ART0001628,-96
409,ART0004363,-114
410,ART0001154,-176
411,ART0003610,-176


In [643]:
sqldf('''select sum(Quantity_x-Quantity_y) as DiffQuantity
        from vendite
        ''')

,DiffQuantity
0,2105


##### creazione colonne mix, volume, prezzo unitario

In [644]:
venditeBudgetS = sqldf('''select NrArticolo, (Quantity*100000000000/7224) as Mix, 7224 as VolumeTotale, TotaleEuro/Quantity as PrezzoUnitario
        from venditeBudgetArt''')
venditeBudgetS

,NrArticolo,Mix,VolumeTotale,PrezzoUnitario
0,ART0000018,27685492,7224,42.690447
1,ART0000041,55370985,7224,90.124277
2,ART0000042,55370985,7224,142.301489
3,ART0000043,27685492,7224,109.097809
4,ART0000115,166112956,7224,163.750000
...,...,...,...,...
408,ART0005109,55370985,7224,26.562945
409,ART0005110,27685492,7224,47.433830
410,ART0005111,27685492,7224,33.203681
411,ART0005112,27685492,7224,42.690447


In [645]:
sqldf('select sum(Mix*VolumeTotale*PrezzoUnitario)/100000000000 from venditeBudgetS')

,sum(Mix*VolumeTotale*PrezzoUnitario)/100000000000
0,395335.890784


In [646]:
sqldf('select sum(Quantity) from venditeConsuntivo')

,sum(Quantity)
0,9329


In [647]:
venditeConsuntivoS = sqldf('''select NrArticolo, (Quantity*100000000000/9329) as Mix, 9329 as VolumeTotale, TotaleEuro/Quantity as PrezzoUnitario
        from venditeConsuntivoArt''')
venditeConsuntivoS

,NrArticolo,Mix,VolumeTotale,PrezzoUnitario
0,ART0000018,21438525,9329,45.860085
1,ART0000041,42877050,9329,100.058367
2,ART0000042,42877050,9329,150.087551
3,ART0000043,21438525,9329,125.072959
4,ART0000115,32157787,9329,365.000000
...,...,...,...,...
408,ART0005109,96473362,9329,23.346952
409,ART0005110,96473362,9329,48.361544
410,ART0005111,75034837,9329,29.183690
411,ART0005112,75034837,9329,37.521888


In [648]:
sqldf('select sum(Mix*VolumeTotale*PrezzoUnitario)/100000000000 from venditeConsuntivoS')

,sum(Mix*VolumeTotale*PrezzoUnitario)/100000000000
0,507842.849547


### Calcolo socostamenti di volume, mix e prezzo

In [649]:
venditeS = pd.merge(venditeConsuntivoS, venditeBudgetS, on = ['NrArticolo'], how = 'outer')
venditeS

,NrArticolo,Mix_x,VolumeTotale_x,PrezzoUnitario_x,Mix_y,VolumeTotale_y,PrezzoUnitario_y
0,ART0000018,21438525,9329,45.860085,27685492,7224,42.690447
1,ART0000041,42877050,9329,100.058367,55370985,7224,90.124277
2,ART0000042,42877050,9329,150.087551,55370985,7224,142.301489
3,ART0000043,21438525,9329,125.072959,27685492,7224,109.097809
4,ART0000115,32157787,9329,365.000000,166112956,7224,163.750000
...,...,...,...,...,...,...,...
408,ART0005109,96473362,9329,23.346952,55370985,7224,26.562945
409,ART0005110,96473362,9329,48.361544,27685492,7224,47.433830
410,ART0005111,75034837,9329,29.183690,27685492,7224,33.203681
411,ART0005112,75034837,9329,37.521888,27685492,7224,42.690447


In [650]:
sqldf('select sum(Mix_x*VolumeTotale_x*PrezzoUnitario_x)/100000000000 from venditeS')

,sum(Mix_x*VolumeTotale_x*PrezzoUnitario_x)/100000000000
0,507842.849547


In [651]:
x = sqldf('select NrArticolo,(Mix_y*VolumeTotale_y*PrezzoUnitario_y)/100000000000 as c from venditeS ')
x

,NrArticolo,c
0,ART0000018,85.380891
1,ART0000041,360.497103
2,ART0000042,569.205951
3,ART0000043,218.195611
4,ART0000115,1964.999990
...,...,...
408,ART0005109,106.251778
409,ART0005110,94.867657
410,ART0005111,66.407360
411,ART0005112,85.380891


In [652]:
BudgetStandardRic = sqldf('select sum(Mix_y*VolumeTotale_y*PrezzoUnitario_y)/100000000000 from venditeS ').iloc[0][0]
MixStandardRic = sqldf('select sum(Mix_y*VolumeTotale_x*PrezzoUnitario_y)/100000000000 from venditeS ').iloc[0][0]
MixEffettivoRic = sqldf('select sum(Mix_x*VolumeTotale_x*PrezzoUnitario_y)/100000000000 from venditeS ').iloc[0][0]
BudgetEffettivoRic = sqldf('select sum(Mix_x*VolumeTotale_x*PrezzoUnitario_x)/100000000000 from venditeS ').iloc[0][0]


In [653]:
print(BudgetStandardRic)
print(MixStandardRic)
print(MixEffettivoRic)
print(BudgetEffettivoRic)

395335.8907839928
510532.7415730725
539112.4259562771
507842.84954721347


In [654]:
ScostamentoVolumeRic = MixStandardRic-BudgetStandardRic
ScostamentoMixRic = MixEffettivoRic-MixStandardRic
ScostamentoPrezzoRic = BudgetEffettivoRic-MixEffettivoRic

In [655]:
print(ScostamentoVolumeRic)
print(ScostamentoMixRic)
print(ScostamentoPrezzoRic)
print(ScostamentoVolumeRic+ScostamentoMixRic+ScostamentoPrezzoRic)

115196.85078907968
28579.684383204556
-31269.576409063593
112506.95876322064


# Vista globale degli scostamenti

In [656]:
data = {'Index': ['Ricavi', 'CostiMP', 'CostiLav', 'Margine'], 
        'Budget' : [BudgetStandardRic, BudgetStandardMP,BudgetStandardLav, (BudgetStandardRic - (BudgetStandardMP+BudgetStandardLav))],
        'ScostamentoVolume' : [ScostamentoVolumeRic,ScostamentoVolumeMP,ScostamentoVolumeLav, (ScostamentoVolumeRic - (ScostamentoVolumeMP+ScostamentoVolumeLav))], 
        'MixStandard' : [MixStandardRic, ImpiegoStandardMP,ImpiegoStandardLav, (MixStandardRic - (ImpiegoStandardMP+ImpiegoStandardLav))],
        'ScostamentoMix' : [ScostamentoMixRic,ScostamentoImpiegoMP,ScostamentoImpiegoLav, (ScostamentoMixRic - (ScostamentoImpiegoMP+ScostamentoImpiegoLav))],
        'MixEffettivo' : [MixEffettivoRic, ImpiegoEffettivoMP,ImpiegoEffettivoLav, (MixEffettivoRic - (ImpiegoEffettivoMP+ImpiegoEffettivoLav))],
        'ScostamentoPrezzo' : [ScostamentoPrezzoRic,ScostamentoPrezzoMP,ScostamentoPrezzoLav, (ScostamentoPrezzoRic - (ScostamentoPrezzoMP+ScostamentoPrezzoLav))], 
        'Consuntivo' : [BudgetEffettivoRic, BudgetEffettivoMP, BudgetEffettivoLav, (BudgetEffettivoRic - (BudgetEffettivoMP+BudgetEffettivoLav))],
        'ScostamentoTotale' : [BudgetEffettivoRic-BudgetStandardRic,BudgetEffettivoMP-BudgetStandardMP,BudgetEffettivoLav-BudgetStandardLav, (BudgetEffettivoRic - BudgetStandardRic - (BudgetEffettivoMP-BudgetStandardMP+BudgetEffettivoLav-BudgetStandardLav))]}

In [657]:
pd.DataFrame(data)

,Index,Budget,ScostamentoVolume,MixStandard,ScostamentoMix,MixEffettivo,ScostamentoPrezzo,Consuntivo,ScostamentoTotale
0,Ricavi,395335.890784,115196.850789,510532.741573,28579.684383,539112.425956,-31269.576409,507842.849547,112506.958763
1,CostiMP,192906.470000,120185.484892,313091.954892,-39991.236636,273100.718257,16538.781743,289639.500000,96733.030000
2,CostiLav,194739.149842,110867.701781,305606.851622,-23700.191552,281906.660070,77518.885803,359425.545873,164686.396031
3,Margine,7690.270942,-115856.335884,-108166.064941,92271.112571,-15894.952370,-125327.243955,-141222.196326,-148912.467268


In [658]:
print(-115856.335884+92271.112571-125327.243955)


-148912.467268


# Repeat ma tenendo conto della valuta

In [659]:
venditeConsuntivoArt = sqldf('''select NrArticolo, sum(Quantity) as Quantity,  sum(ImportoVenditaValutaLocaleTOTALEVENDITA) as TotaleLocale, Valuta, TassoCambioMedio
        from venditeConsuntivo
        group by NrArticolo, Valuta, TassoCambioMedio''')
venditeBudgetArt = sqldf('''select NrArticolo, sum(Quantity) as Quantity, sum(ImportoVenditaValutaLocaleTOTALEVENDITA) as TotaleLocale, Valuta, TassoCambioMedio
        from venditeBudget
        group by NrArticolo, Valuta, TassoCambioMedio ''')

In [660]:
venditeBudgetArt

,NrArticolo,Quantity,TotaleLocale,Valuta,TassoCambioMedio
0,ART0000018,2,90.00,2,1.0541
1,ART0000041,4,380.00,2,1.0541
2,ART0000042,4,600.00,2,1.0541
3,ART0000043,2,230.00,2,1.0541
4,ART0000115,12,1965.00,1,1
...,...,...,...,...,...
410,ART0005111,2,70.00,2,1.0541
411,ART0005112,2,90.00,2,1.0541
412,ART0005136,87,8933.00,1,1
413,ART0005136,80,4175.21,2,1.0541


##### creazione colonne mix, volume, prezzo unitario (locale)

In [661]:
venditeConsuntivoS = sqldf('''select NrArticolo, (Quantity*100000000000/9329) as Mix, 9329 as VolumeTotale, TotaleLocale/Quantity as PrezzoUnitario, Valuta, TassoCambioMedio, (TotaleLocale/Quantity)/TassoCambioMedio as PrezzoUnitarioEuro
        from venditeConsuntivoArt''')
venditeBudgetS = sqldf('''select NrArticolo, (Quantity*100000000000/7224) as Mix, 7224 as VolumeTotale, TotaleLocale/Quantity as PrezzoUnitario, Valuta, TassoCambioMedio, (TotaleLocale/Quantity)/TassoCambioMedio as PrezzoUnitarioEuro
        from venditeBudgetArt''')


### Calcolo socostamenti di volume, mix, prezzo e valuta

In [662]:
venditeS1 = pd.merge(venditeConsuntivoS, venditeBudgetS, on = ['NrArticolo', 'Valuta'], how = 'outer')
venditeS1
#venditeS1.to_excel('venditeS1.xlsx')

,NrArticolo,Mix_x,VolumeTotale_x,PrezzoUnitario_x,Valuta,TassoCambioMedio_x,PrezzoUnitarioEuro_x,Mix_y,VolumeTotale_y,PrezzoUnitario_y,TassoCambioMedio_y,PrezzoUnitarioEuro_y
0,ART0000018,21438525,9329,55.000000,2,1.1993,45.860085,27685492,7224,45.000000,1.0541,42.690447
1,ART0000041,42877050,9329,120.000000,2,1.1993,100.058367,55370985,7224,95.000000,1.0541,90.124277
2,ART0000042,42877050,9329,180.000000,2,1.1993,150.087551,55370985,7224,150.000000,1.0541,142.301489
3,ART0000043,21438525,9329,150.000000,2,1.1993,125.072959,27685492,7224,115.000000,1.0541,109.097809
4,ART0000115,32157787,9329,365.000000,1,1,365.000000,166112956,7224,163.750000,1,163.750000
...,...,...,...,...,...,...,...,...,...,...,...,...
410,ART0005111,75034837,9329,35.000000,2,1.1993,29.183690,27685492,7224,35.000000,1.0541,33.203681
411,ART0005112,75034837,9329,45.000000,2,1.1993,37.521888,27685492,7224,45.000000,1.0541,42.690447
412,ART0005136,686032800,9329,39.109375,1,1,39.109375,1204318936,7224,102.678161,1,102.678161
413,ART0005136,439489763,9329,229.121951,2,1.1993,191.046403,1107419712,7224,52.190125,1.0541,49.511550


In [663]:
BudgetStandardRic = sqldf('select sum(Mix_y*VolumeTotale_y*PrezzoUnitario_y/TassoCambioMedio_y)/100000000000 from venditeS1 ').iloc[0][0]
MixStandardRic = sqldf('select sum(Mix_y*VolumeTotale_x*PrezzoUnitario_y/TassoCambioMedio_y)/100000000000 from venditeS1 ').iloc[0][0]
MixEffettivoRic = sqldf('select sum(Mix_x*VolumeTotale_x*PrezzoUnitario_y/TassoCambioMedio_y)/100000000000 from venditeS1 ').iloc[0][0]
BudgetEffettivoRic = sqldf('select sum(Mix_x*VolumeTotale_x*PrezzoUnitario_x/TassoCambioMedio_y)/100000000000 from venditeS1 ').iloc[0][0]
BudgetEffettivoValutaRic = sqldf('select sum(Mix_x*VolumeTotale_x*PrezzoUnitario_x/TassoCambioMedio_x)/100000000000 from venditeS1 ').iloc[0][0]

In [664]:
print(BudgetStandardRic)
print(MixStandardRic)
print(MixEffettivoRic)
print(BudgetEffettivoRic)
print(BudgetEffettivoValutaRic)

395335.8907780576
510532.74156540784
513440.445133813
569389.7443994995
507842.8495423171


In [665]:
ScostamentoVolumeRic = MixStandardRic-BudgetStandardRic
ScostamentoMixRic = MixEffettivoRic-MixStandardRic
ScostamentoPrezzoRic = BudgetEffettivoRic-MixEffettivoRic
ScostamentoValutaRic = BudgetEffettivoValutaRic-BudgetEffettivoRic

In [666]:
print(ScostamentoVolumeRic)
print(ScostamentoMixRic)
print(ScostamentoPrezzoRic)
print(ScostamentoValutaRic)
print(ScostamentoVolumeRic+ScostamentoMixRic+ScostamentoPrezzoRic+ScostamentoValutaRic)

115196.85078735021
2907.7035684051807
55949.29926568648
-61546.8948571824
112506.95876425947


# Vista globale degli scostamenti v2

In [667]:
data = {'Index': ['Ricavi', 'CostiMP', 'CostiLav', 'Margine'], 
        'Budget' : [BudgetStandardRic, BudgetStandardMP,BudgetStandardLav, (BudgetStandardRic - (BudgetStandardMP+BudgetStandardLav))],
        'ScostamentoVolume' : [ScostamentoVolumeRic,ScostamentoVolumeMP,ScostamentoVolumeLav, (ScostamentoVolumeRic - (ScostamentoVolumeMP+ScostamentoVolumeLav))], 
        'MixStandard' : [MixStandardRic, ImpiegoStandardMP,ImpiegoStandardLav, (MixStandardRic - (ImpiegoStandardMP+ImpiegoStandardLav))],
        'ScostamentoMix' : [ScostamentoMixRic,ScostamentoImpiegoMP,ScostamentoImpiegoLav, (ScostamentoMixRic - (ScostamentoImpiegoMP+ScostamentoImpiegoLav))],
        'MixEffettivo' : [MixEffettivoRic, ImpiegoEffettivoMP,ImpiegoEffettivoLav, (MixEffettivoRic - (ImpiegoEffettivoMP+ImpiegoEffettivoLav))],
        'ScostamentoPrezzo' : [ScostamentoPrezzoRic,ScostamentoPrezzoMP,ScostamentoPrezzoLav, (ScostamentoPrezzoRic - (ScostamentoPrezzoMP+ScostamentoPrezzoLav))], 
        'MixValuta' : [BudgetEffettivoRic, BudgetEffettivoMP,BudgetEffettivoLav, (BudgetEffettivoRic - (BudgetEffettivoMP+BudgetEffettivoLav))],
        'ScostamentoValuta' : [ScostamentoValutaRic, 0, 0, (ScostamentoValutaRic - (0+0))],
        'Consuntivo' : [BudgetEffettivoValutaRic, BudgetEffettivoMP, BudgetEffettivoLav, (BudgetEffettivoValutaRic - (BudgetEffettivoMP+BudgetEffettivoLav))],
        'ScostamentoTotale' : [BudgetEffettivoValutaRic-BudgetStandardRic,BudgetEffettivoMP-BudgetStandardMP,BudgetEffettivoLav-BudgetStandardLav, (BudgetEffettivoValutaRic - BudgetStandardRic - (BudgetEffettivoMP-BudgetStandardMP+BudgetEffettivoLav-BudgetStandardLav))]}

In [668]:
pd.DataFrame(data)

,Index,Budget,ScostamentoVolume,MixStandard,ScostamentoMix,MixEffettivo,ScostamentoPrezzo,MixValuta,ScostamentoValuta,Consuntivo,ScostamentoTotale
0,Ricavi,395335.890778,115196.850787,510532.741565,2907.703568,513440.445134,55949.299266,569389.744399,-61546.894857,507842.849542,112506.958764
1,CostiMP,192906.470000,120185.484892,313091.954892,-39991.236636,273100.718257,16538.781743,289639.500000,0.000000,289639.500000,96733.030000
2,CostiLav,194739.149842,110867.701781,305606.851622,-23700.191552,281906.660070,77518.885803,359425.545873,0.000000,359425.545873,164686.396031
3,Margine,7690.270936,-115856.335885,-108166.064949,66599.131756,-41566.933193,-38108.368281,-79675.301473,-61546.894857,-141222.196331,-148912.467267


In [331]:
print(-2508.715616-48377.117694-38034.622697-61546.894857)


-150467.350864


come vedremo, i volumi di vendita non coincidono ai volumi di produzione, come negli esercizi fatti a lezione.

Invece, a budget, si vendenono circa 412 prodotti diversi, mentre si lavorano circa 503, dei quali soltanto 73 passano per una fase di montaggio nell'anno in quesitone.

Questo ci porta alla conclusione che ci sono delle rimanenze, e non tutto cio che e prodotto nell'anno e venduto nello stesso anno.

In [332]:
impiegoBudget

,NrArticolo,Descrizione,NrAreaProduzione,TempoRisorsa,ImpiegoUnitario,CostoOrario,QuantitydiOutput
0,ART0000018,Controllo qualità,CQ,0.00,0.000000,12.750000,2.0
1,ART0000018,Fresatura,A30,0.75,0.375000,51.000000,2.0
2,ART0000018,Tornitura,A20,1.00,0.500000,68.000000,2.0
3,ART0000041,Controllo qualità,CQ,0.00,0.000000,12.750000,4.0
4,ART0000041,Fresatura,A30,1.25,0.312500,51.000000,4.0
...,...,...,...,...,...,...,...
1222,ART0005136,Foratura/Maschiatura,A32,3.00,NaN,42.500000,0.0
1223,ART0005136,Fresatura,A30,21.33,0.888750,47.600000,24.0
1224,ART0005136,Montaggio,A40,8.25,NaN,42.500000,0.0
1225,ART0005136,Taglio spezzoni,A20,4.50,0.155172,68.000000,29.0


In [ ]:
sqldf('select NrArticolo, QuantitydiOutput from impiegoBudget group by NrArticolo,QuantitydiOutput order by NrArticolo')

,NrArticolo,QuantitydiOutput
0,ART0000018,2.00
1,ART0000041,4.00
2,ART0000042,4.00
3,ART0000043,2.00
4,ART0000043,2.25
...,...,...
645,ART0005136,24.00
646,ART0005136,29.00
647,ART0005136,34.00
648,ART0005136,283.00


In [ ]:
sqldf('select NrArticolo,Quantity from venditeBudget group by NrArticolo,Quantity order by NrArticolo')

,NrArticolo,Quantity
0,ART0000018,2
1,ART0000041,4
2,ART0000042,4
3,ART0000043,2
4,ART0000115,1
...,...,...
585,ART0005136,30
586,ART0005136,32
587,ART0005136,42
588,ART0005136,50


In [335]:
sqldf('select NrArticolo,sum(Quantity) as quantita from venditeBudget group by NrArticolo order by NrArticolo')

,NrArticolo,quantita
0,ART0000018,2
1,ART0000041,4
2,ART0000042,4
3,ART0000043,2
4,ART0000115,12
...,...,...
408,ART0005109,4
409,ART0005110,2
410,ART0005111,2
411,ART0005112,2


In [334]:
sqldf("SELECT DISTINCT NrArticolo FROM dfImpiegoRisorse WHERE (budgetConsuntivo = 'BUDGET' or budgetConsuntivo = 'Budget')")

,NrArticolo
0,ART0000018
1,ART0000041
2,ART0000042
3,ART0000043
4,ART0000115
...,...
499,ART0005109
500,ART0005110
501,ART0005111
502,ART0005112


In [255]:
sqldf("SELECT DISTINCT NrArticolo,QuantitydiOutput FROM dfImpiegoRisorse WHERE (budgetConsuntivo = 'BUDGET' or budgetConsuntivo = 'Budget') and Descrizione = 'Montaggio'")

,NrArticolo,QuantitydiOutput
0,ART0000748,50.0
1,ART0000748,10.0
2,ART0000748,30.0
3,ART0000752,1.0
4,ART0000752,6.0
...,...,...
69,ART0004099,1.0
70,ART0004104,4.0
71,ART0004146,31.0
72,ART0004146,0.0


In [336]:
pd.merge(
    sqldf('select NrArticolo,sum(Quantity) as quantita from venditeBudget group by NrArticolo order by NrArticolo'),
    sqldf("SELECT DISTINCT NrArticolo,QuantitydiOutput FROM dfImpiegoRisorse WHERE (budgetConsuntivo = 'BUDGET' or budgetConsuntivo = 'Budget') and Descrizione = 'Montaggio'"), 
    on='NrArticolo', how='outer')

,NrArticolo,quantita,QuantitydiOutput
0,ART0000018,2,NaN
1,ART0000041,4,NaN
2,ART0000042,4,NaN
3,ART0000043,2,NaN
4,ART0000115,12,NaN
...,...,...,...
439,ART0005109,4,NaN
440,ART0005110,2,NaN
441,ART0005111,2,NaN
442,ART0005112,2,NaN
